# Evaluate

In this notebook we evaluate the accuracy of the predicted alignments.

In [1]:
%matplotlib inline

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import os.path
import pandas as pd
import pickle
import re
from plotnine import *
from pandas.api.types import CategoricalDtype

In [5]:
DATASET = 'train' # 'test'
VERSION = 'full'

In [6]:
ANNOTATIONS_ROOT = Path('/home/ijain/ttmp/Chopin_Mazurkas/annotations_beat')
query_list = Path(f'cfg_files/queries.{DATASET}.{VERSION}')

### Evaluate hypothesis directory

In [5]:
def eval_dir(hypdir, querylist, annot_root_1, annot_root_2, hop_sec, savefile = None):
    allErrs = {}
    cnt = 0
    print(f'Processing {hypdir} ', end='')
    with open(querylist, 'r') as f:
        for line in f:
            parts = line.strip().split()
            assert len(parts) == 2
            basename = os.path.basename(parts[0]) + '__' + os.path.basename(parts[1])
            hypfile = hypdir + '/' + basename + '.pkl'
            if not os.path.exists(hypfile):
                print("X", end='')
                continue
            err = eval_file(hypfile, annot_root_1, annot_root_2, hop_sec)
            if err is not None:
                allErrs[basename] = err
            cnt += 1
            if cnt % 500 == 0:
                print(".", end='')
    print(' done')
    if savefile:
        pickle.dump(allErrs, open(savefile, 'wb'))
        
    return allErrs

In [6]:
def eval_file(hypfile, annot_root_1, annot_root_2, hop_sec):
    parts = os.path.basename(hypfile).split('__')
    assert len(parts) == 2
    piece = extractPieceName(parts[0])
    annotfile1 = (annot_root_1 / piece / parts[0]).with_suffix('.beat')
    annotfile2 = (annot_root_2 / piece / parts[1]).with_suffix('.beat')
    
    # if groundtruth annotation files are empty, skip this hypothesis file
    try:
        gt1, gt2 = getTimestamps(annotfile1, annotfile2)
        hypalign = loadAlignment(hypfile) # warping path in frames
    except:
        print(f'Skipping hypothesis file {hypfile}')
        return None

    if hypalign is None:
        err = np.array(np.ones(gt1.shape) * np.inf)
    else:
        pred2 = np.interp(gt1, hypalign[0,:]*hop_sec, hypalign[1,:]*hop_sec)
        err = pred2 - gt2
    return err

In [7]:
def extractPieceName(fullpath):
    basename = os.path.basename(fullpath) # e.g. Chopin_Op068No3_Sztompka-1959_pid9170b-21
    parts = basename.split('_')
    piece = '_'.join(parts[0:2]) # e.g. Chopin_Op068No3
    return piece

In [8]:
def getTimestamps(annotfile1, annotfile2):
    df1 = pd.read_csv(annotfile1, header=None, sep='\s+', skiprows=3) 
    df2 = pd.read_csv(annotfile2, header=None, sep='\s+', skiprows=3)

    df_merged = pd.merge(df1, df2, on=[2], how='inner')
    df_merged = df_merged[df_merged[2].astype(str).str.match(".*\d$")]

    return df_merged['0_x'], df_merged['0_y']

In [9]:
def loadAlignment(hypfile):
    with open(hypfile, 'rb') as f:
        d = pickle.load(f)
    return d

In [10]:
def eval_all_dirs(rootdir, querylist, hop_sec, outdir, system, benchmark):
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if benchmark == 'partialOverlap':
        annot_root_1 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Benchmarks/partialStart/annotations_beat')
        annot_root_2 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Benchmarks/partialEnd/annotations_beat')
    elif 'prepost' in benchmark:
        sec = benchmark.split('_')[-1]
        annot_root_1 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Benchmarks/pre_{sec}/annotations_beat')
        annot_root_2 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Benchmarks/post_{sec}/annotations_beat')
    elif benchmark == 'matching':
        annot_root_1 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Modified/annotations_beat')
        annot_root_2 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Modified/annotations_beat')
    else:
        annot_root_1 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Benchmarks/{benchmark}/annotations_beat')
        annot_root_2 = Path(f'/home/ijain/ttmp/Chopin_Mazurkas_Modified/annotations_beat')
    for hypdir in glob.glob(f'{rootdir}/{benchmark}/{system}'):
        outpath = outdir + '/' + f'{benchmark}'
        Path(outpath).mkdir(parents=True, exist_ok=True)
        savefile = outpath + '/' + os.path.basename(hypdir) + '.pkl'
        allErrs = eval_dir(hypdir, querylist, annot_root_1, annot_root_2, hop_sec, savefile = savefile)

**Evaluate on Data**

In [11]:
EXPERIMENTS_ROOT = f'experiments_{DATASET}/{VERSION}'
hop_sec = 512 * 1 / 22050
outdir = f'evaluations_{DATASET}/{VERSION}'
Path(outdir).mkdir(parents=True, exist_ok=True)

In [12]:
def eval_benchmark(experiments_root, hop_sec, outdir, system, benchmark):
    eval_all_dirs(EXPERIMENTS_ROOT, query_list, hop_sec, outdir, system, benchmark)

In [7]:
# SYSTEMS = ['DTW1', 'DTW2', 'DTW3', 'SubseqDTW1', 'SubseqDTW2', 'SubseqDTW3', 'NWTW', 'FlexDTW', 'parflex']
SYSTEMS = ['sparse_parflex', 'parflex']
BENCHMARKS = ['Matching', 'Subseq_20', 'Subseq_30', 'Subseq_40', 'PartialStart', 'PartialEnd', 'PartialOverlap', 
              'Pre_5', 'Pre_10', 'Pre_20', 'Post_5', 'Post_10', 'Post_20', 'PrePost_5', 
              'PrePost_10', 'PrePost_20']

In [8]:
systems = [system.lower() for system in SYSTEMS]
benchmarks = []
for benchmark in BENCHMARKS:
    if benchmark == 'PartialOverlap':
        benchmarks.append('partialOverlap')
    elif benchmark == 'PartialStart':
        benchmarks.append('partialStart')
    elif benchmark == 'PartialEnd':
        benchmarks.append('partialEnd')
    elif 'PrePost' in benchmark:
        sec = benchmark.split('_')[-1]
        benchmarks.append(f'prepost_{sec}')
    else:
        benchmarks.append(benchmark.lower())

In [15]:
def eval_all_benchmarks(experiments_root, hop_sec, outdir, systems, benchmarks):
    for benchmark in benchmarks:
        for system in systems:
            eval_benchmark(experiments_root, hop_sec, outdir, system, benchmark)          

In [16]:
eval_all_benchmarks(EXPERIMENTS_ROOT, hop_sec, outdir, systems, benchmarks)

Processing experiments_train/full/matching/sparse_parflex .......XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.... done
Processing experiments_train/full/matching/parflex .Skipping hypothesis file experiments_train/full/matching/parflex/Chopin_Op063No3_Cortot-1951_pid9066b-12__Chopin_Op063No3_Magaloff-1978_pid9074g-12.pkl
Skipping hypothesis file experiments_train/full/matching/parflex/Chopin_Op063No3_Zak-1937_pid9085-12__Chopin_Op063No3_Kissin-1993_pid9056-07.pkl
Skipping hypothesis file experiments_train/full/matching/parflex/Chopin_Op063No3_Zak-1937_pid9085-12__Chopin_Op063No3_Sokolov-2002_pid610633-06.pkl
...Skipping hypothesis file experiments_train/full/matching/parflex/Chopin_Op063No3_Malcuzynski-1961_pid918904-12__Chopin_Op063No3_Rosenthal-1931_pid9141-09.pkl
Skipping hypothesis file experiments_train/full/matching/parflex/Chopin_Op063No3_Lushtak-2004_pid9088-22__Chopin_Op063No3_Rosenthal-1930_pid9183-05.pkl
...XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

### Plot error vs tolerance

In [17]:
def calc_error_rates(errFile, maxTol):
    print(errFile)
    # read from file
    with open(errFile, 'rb') as f:
        allErrs = pickle.load(f)
    
    # collect all errors
    errsFlat = []
    # print(allErrs)
    for query in allErrs:
        # print(query)
        errs = np.array(allErrs[query])
        errsFlat.append(errs)
    # print(errsFlat)
    errsFlat = np.concatenate(errsFlat)
    
    # calculate error rates
    errRates = np.zeros(maxTol+1)
    for i in range(maxTol+1):
        errRates[i] = np.mean(np.abs(errsFlat) > i/1000)
    
    return errRates, errsFlat

In [18]:
def calc_error_rates_batch(indir, basenames, maxTol):
    errRates = np.zeros((len(basenames), maxTol+1))
    allErrVals = []
    print('Computing error rates ', end='')
    for i, basename in enumerate(basenames):
        errFile = indir + '/' + basename + '.pkl'
        errRates[i,:], errors = calc_error_rates(errFile, maxTol)
        allErrVals.append(errors)
        print('.', end='')
    print(' done')
    return errRates, allErrVals

In [19]:
def plot_multiple_roc(errRates, basenames):
    numSystems = errRates.shape[0]
    maxTol = errRates.shape[1] - 1
    for i in range(numSystems):
        plt.plot(np.arange(maxTol+1), errRates[i,:] * 100.0)
    plt.legend(basenames, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Error Tolerance (ms)')
    plt.ylabel('Error Rate (%)')
    plt.show()
    return

**Evaluate on Data**

In [20]:
EVAL_ROOT_DIR = f'evaluations_{DATASET}/{VERSION}'
toPlot = []

for benchmark in benchmarks:
    for system in systems:
        toPlot.append(f'{benchmark}/{system}')
maxTol = 1000 # in msec

In [21]:
def get_errs(eval_root_dir, toPlot, maxTol):
    errRates, errVals = calc_error_rates_batch(EVAL_ROOT_DIR, toPlot, maxTol)
    return errRates, errVals

In [22]:
errRates, errVals = get_errs(EVAL_ROOT_DIR, toPlot, maxTol)

Computing error rates evaluations_train/full/matching/sparse_parflex.pkl
.evaluations_train/full/matching/parflex.pkl
.evaluations_train/full/subseq_20/sparse_parflex.pkl
.evaluations_train/full/subseq_20/parflex.pkl
.evaluations_train/full/subseq_30/sparse_parflex.pkl
.evaluations_train/full/subseq_30/parflex.pkl
.evaluations_train/full/subseq_40/sparse_parflex.pkl
.evaluations_train/full/subseq_40/parflex.pkl
.evaluations_train/full/partialStart/sparse_parflex.pkl
.evaluations_train/full/partialStart/parflex.pkl
.evaluations_train/full/partialEnd/sparse_parflex.pkl
.evaluations_train/full/partialEnd/parflex.pkl
.evaluations_train/full/partialOverlap/sparse_parflex.pkl
.evaluations_train/full/partialOverlap/parflex.pkl
.evaluations_train/full/pre_5/sparse_parflex.pkl
.evaluations_train/full/pre_5/parflex.pkl
.evaluations_train/full/pre_10/sparse_parflex.pkl
.evaluations_train/full/pre_10/parflex.pkl
.evaluations_train/full/pre_20/sparse_parflex.pkl
.evaluations_train/full/pre_20/parfl

In [23]:
errRates.shape[0] == len(SYSTEMS) * len(BENCHMARKS)

True

In [24]:
np.save(f'evaluations_{DATASET}/{VERSION}_errRates', errRates)

### Make Plots (New)

In [2]:
def make_df(time, errRates):
    data = {}
    
    num_systems = len(SYSTEMS)
    num_benchmarks = len(BENCHMARKS)
    
    data['Benchmark'] = []
    for benchmark in BENCHMARKS:
        data['Benchmark'] += [benchmark] * num_systems
    
    data['System'] = [system for system in SYSTEMS] * num_benchmarks

    data['Error'] = []
    for i in range(num_benchmarks*num_systems):
            data['Error'].append(errRates[i][time]*100)
            
    df = pd.DataFrame.from_dict(data)
    benchmark_categories = CategoricalDtype(categories=BENCHMARKS, ordered=True)
    df.Benchmark = df.Benchmark.astype(benchmark_categories)
    system_categories = CategoricalDtype(categories=SYSTEMS, ordered=True)
    df.System = df.System.astype(system_categories)
    
    return df

In [9]:
errRates = np.load(f'evaluations_{DATASET}/{VERSION}_errRates.npy')

In [10]:
errRates = np.load(f'evaluations_{DATASET}/{VERSION}_errRates.npy')

In [11]:
# Error rates at 200ms tolerance
ms200_df = make_df(200, errRates)

In [12]:
# Error rates at 100ms and 500ms tolerance (black horizontal bars on plot)
ms100_df = make_df(100, errRates)
ms500_df = make_df(500, errRates)

In [13]:
colors = ['#620FA2', '#874CB4', '#A77DC4', '#0C7715', '#229E25', '#3AC738', '#FF0000', '#16418F']

In [14]:
(ggplot(ms200_df, aes(x="System", y="Error", fill="System")) +
    geom_bar(width=0.7,
             position=position_dodge2(preserve='single', width=0.25),
             stat='identity') +
    scale_y_continuous(expand=(0, 0), limits=(0, 100)) +
    scale_fill_manual(values=colors) +
    geom_crossbar(aes(ymin="Error", ymax="Error"),
                  data=ms100_df, width=0.5, color="black") +
    geom_crossbar(aes(ymin="Error", ymax="Error"),
                  data=ms500_df, width=0.5, color="black") +
    facet_grid('. ~ Benchmark') +
    theme_bw() +
    labs(y="Error Rate (%)") +
    theme(
        dpi=600,
        figure_size=(15, 5),
        legend_position='bottom',
        legend_direction="horizontal",
        legend_box="horizontal",
        legend_background=element_blank(),
        legend_title=element_text(size=10),
        legend_text=element_text(size=8),
        strip_background=element_blank(),
        strip_text_x=element_text(size=9, weight='bold'),
        axis_text_x=element_blank(),
        axis_ticks_x=element_blank(),
        axis_text_y=element_text(size=6, colour='black'),
        axis_title_x=element_blank(),
        axis_title_y=element_text(size=10, margin={'r': 6.0})
    ) +
    guides(fill=guide_legend(
        nrow=1, byrow=True, title="",
        override_aes={'size': 0}
    ))
)


In [15]:
(ggplot(ms200_df, aes(x="System", y="Error", fill="System")) +
    geom_bar(width=0.7,
             position=position_dodge2(preserve='single', width=0.25),
             stat='identity') +
    scale_y_continuous(expand=(0, 0), limits=(0, 100)) +
    scale_fill_manual(values=colors) +
    geom_crossbar(aes(ymin="Error", ymax="Error"),
                  data=ms100_df, width=0.5, color="black") +
    geom_crossbar(aes(ymin="Error", ymax="Error"),
                  data=ms500_df, width=0.5, color="black") +
    facet_grid('. ~ Benchmark') +
    theme_bw() +
    labs(y="Error Rate (%)") +
    theme(
        dpi=600,
        figure_size=(15, 5),
        legend_position='bottom',
        legend_direction="horizontal",
        legend_box="horizontal",
        legend_background=element_blank(),
        legend_title=element_text(size=10),
        legend_text=element_text(size=8),
        strip_background=element_blank(),
        strip_text_x=element_text(size=9, weight='bold'),
        axis_text_x=element_blank(),
        axis_ticks_x=element_blank(),
        axis_text_y=element_text(size=6, colour='black'),
        axis_title_x=element_blank(),
        axis_title_y=element_text(size=10, margin={'r': 6.0})
    ) +
    guides(fill=guide_legend(
        nrow=1, byrow=True, title="",
        override_aes={'size': 0}
    ))
)


In [33]:
ms200_df.to_csv('evaluations.csv')

In [16]:
pd.DataFrame(ms200_df.pivot(index='Benchmark', columns='System', values='Error')['parflex'])

,parflex
Benchmark,
Matching,9.259289
Subseq_20,11.121317
Subseq_30,10.358704
Subseq_40,9.564541
PartialStart,7.226481
PartialEnd,9.475420
PartialOverlap,7.500250
Pre_5,9.314910
Pre_10,9.231552


In [17]:
import pandas as pd
ms200_df = pd.read_csv('evaluations.csv')

In [18]:
ms500_df

,Benchmark,System,Error
0,Matching,sparse_parflex,4.836059
1,Matching,parflex,4.838939
2,Subseq_20,sparse_parflex,91.472993
3,Subseq_20,parflex,7.162297
4,Subseq_30,sparse_parflex,83.252730
5,Subseq_30,parflex,6.024459
6,Subseq_40,sparse_parflex,80.001915
7,Subseq_40,parflex,5.835557
8,PartialStart,sparse_parflex,15.067538
9,PartialStart,parflex,3.421055


In [19]:
ms100_df

,Benchmark,System,Error
0,Matching,sparse_parflex,18.872160
1,Matching,parflex,18.874854
2,Subseq_20,sparse_parflex,93.048144
3,Subseq_20,parflex,19.962772
4,Subseq_30,sparse_parflex,86.216336
5,Subseq_30,parflex,19.821548
6,Subseq_40,sparse_parflex,82.840116
7,Subseq_40,parflex,18.360358
8,PartialStart,sparse_parflex,27.028775
9,PartialStart,parflex,16.542174


In [20]:
x = ms200_df.pivot(index='Benchmark', columns='System', values='Error')

In [21]:
x['diff'] = x['sparse_parflex'] - x['parflex']

In [22]:
x

System,parflex,sparse_parflex,diff
Benchmark,,,
Matching,9.259289,9.256135,-0.003154
PartialEnd,9.475420,9.991178,0.515757
PartialOverlap,7.500250,46.302557,38.802307
PartialStart,7.226481,18.679824,11.453343
Post_10,9.203724,9.203724,0.000000
Post_20,9.216390,9.216390,0.000000
Post_5,9.209339,9.209339,0.000000
PrePost_10,9.287831,9.287831,0.000000
PrePost_20,9.252142,9.252142,0.000000
